# Debug notebook
This notebook is only used to troubleshoot the S2 ARD workflow processor.

In [6]:
import rediswq

host="redis-master"
# Uncomment next two lines if you do not have Kube-DNS working.
# import os
# host = os.getenv("REDIS_SERVICE_HOST")

q = rediswq.RedisWQ(name="jobS2", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: a365d79a-d176-4bac-999d-d965fb536dba
Initial queue state: empty=False


In [9]:
import json
from utils.prepS2 import prepareS2

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    print("%s: %s" % (x, loaded_json[x]))
    #prepareS2(loaded_json["in_scene"], loaded_json["out_dir"], inter_dir=loaded_json["inter_dir"], prodlevel=loaded_json["prodlevel"], source=loaded_json["source"])
    prepareS2(loaded_json["in_scene"], inter_dir=loaded_json["inter_dir"])

In [10]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

Working on {"in_scene": "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE", "out_dir": "public-eo-data", "inter_dir": "/data/inter", "prodlevel": "L2A", "source": "gcloud"}
in_scene: S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE
out_dir: public-eo-data
inter_dir: /data/inter
prodlevel: L2A
source: gcloud


In [ ]:
print("Queue empty!")